In [1]:
import cv2
import mediapipe as mp
import socket

# MediaPipe Hands 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 웹캠 설정
cap = cv2.VideoCapture(0)

# TCP 서버 설정
def send_hand_landmarks(host='127.0.0.1', port=6521):
    # 소켓 객체 생성
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        # 서버에 연결
        client_socket.connect((host, port))
        print(f"서버에 연결되었습니다. {host}:{port}")

        while True:
            # 웹캠에서 프레임 읽기
            success, image = cap.read()
            if not success:
                continue

            # 이미지를 RGB로 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # 성능을 위해 이미지 쓰기 불가능으로 설정
            image.flags.writeable = False

            # MediaPipe를 통해 손 검출
            results = hands.process(image)

            # 이미지를 RGB에서 BGR로 변환하여 그리기 작업 준비
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # 검출된 손의 랜드마크를 이미지에 그림
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2))
                    
                    # 엄지 끝점 랜드마크 가져오기
                    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                    # 검지 끝점 랜드마크 가져오기
                    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                    # 손목 랜드마크 가져오기
                    wrist_landmark = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
                    
                    # 랜드마크들의 좌표 값을 소수점 셋째 자리까지 반올림하여 문자열로 변환하여 서버로 전송
                    landmarks_str = f"{round(thumb_tip.x, 3)},{round(thumb_tip.y, 3)},{round(thumb_tip.z, 3)};" \
                                    f"{round(index_tip.x, 3)},{round(index_tip.y, 3)},{round(index_tip.z, 3)};" \
                                    f"{round(wrist_landmark.x, 3)},{round(wrist_landmark.y, 3)},{round(wrist_landmark.z, 3)};"
                    print(landmarks_str)
                    client_socket.sendall(landmarks_str.encode())

            # 결과 이미지 표시
            cv2.imshow('Hand Tracking', image)

            # 'q'를 눌러 종료
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    send_hand_landmarks()

서버에 연결되었습니다. 127.0.0.1:6521
0.166,0.76,-0.021;0.154,0.791,-0.039;0.069,0.995,-0.0;
0.199,0.694,-0.053;0.156,0.754,-0.074;0.119,0.965,-0.0;
0.23,0.641,-0.012;0.211,0.672,-0.02;0.148,0.916,-0.0;
0.253,0.568,0.007;0.232,0.624,-0.001;0.166,0.858,-0.0;
0.274,0.52,-0.002;0.26,0.569,-0.005;0.181,0.801,-0.0;
0.292,0.474,-0.003;0.276,0.538,0.001;0.197,0.751,-0.0;
0.304,0.428,0.004;0.295,0.486,0.007;0.212,0.7,-0.0;
0.349,0.294,-0.007;0.352,0.309,-0.01;0.252,0.519,-0.0;
0.343,0.305,0.028;0.337,0.299,0.051;0.226,0.603,0.0;
0.357,0.273,0.014;0.361,0.278,0.015;0.247,0.541,0.0;
0.365,0.254,-0.008;0.363,0.269,-0.022;0.262,0.499,-0.0;
0.376,0.237,-0.001;0.374,0.247,-0.013;0.269,0.482,0.0;
0.385,0.224,0.001;0.383,0.234,-0.013;0.278,0.472,0.0;
0.392,0.218,-0.003;0.388,0.224,-0.014;0.281,0.465,0.0;
0.396,0.212,-0.007;0.393,0.22,-0.022;0.286,0.457,0.0;
0.405,0.214,-0.012;0.399,0.218,-0.025;0.286,0.456,0.0;
0.413,0.218,-0.013;0.403,0.22,-0.024;0.286,0.455,0.0;
0.405,0.219,-0.008;0.4,0.219,-0.022;0.282,0.456